<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
##### Your Code Here #####
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./data/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

In [3]:
#df.iloc[:,1:-1].head()

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### Notes on data for each column

##### Drop customer ID (doesn't add any helpful variance/information)

##### Encode binary columns to 0,1
- gender
- Partner
- dependents
- Phone Service
- Paperless Billing

##### MultipleLines has only 3 values
- No, Yes, No phone service

##### InternetService has only 3 values
- Fiber Optic, DSL, No

##### These columns all have 3 values that are all the same options ( No, Yes, No internet service )
- OnlineSecurity
- OnlineBackup
- DeviceProtection
- TechSupport
- StreamingTV
- StreamingMovies

##### Contract has 3 values
- Month-to-month, two year, one year

##### PaymentMethod has 4 values
- Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)

##### These columns are numeric - We will want to normalize them
- tenure
- MonthlyCharges
- TotalCharges

##### TotalCharges has blank values
 - Based on the cell below, it looks like this is missing information and not a value of 0

## Plan for cleaning data

- encode binaries as 1 or 0
- fill the blanks in total charges with the median
- one hot encode the rest of the categoricals
- normalize the 3 numeric columns


In [4]:
# Let's look at the sorted list of possible values 
df['TotalCharges'].value_counts(normalize=True).index.sort_values()

Index([' ', '100.2', '100.25', '100.35', '100.4', '100.8', '100.9', '1001.2',
       '1001.5', '1003.05',
       ...
       '995.35', '996.45', '996.85', '996.95', '997.65', '997.75', '998.1',
       '999.45', '999.8', '999.9'],
      dtype='object', length=6531)

In [5]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [6]:
train = df.iloc[:,1:-1]
train.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65


In [7]:
# convert the binary columns to 0's and 1's

from sklearn.preprocessing import LabelBinarizer

for col in train.columns:
    if len(train[col].value_counts()) == 2:
        lb = LabelBinarizer()
        train[col] = lb.fit_transform(train[col])
        
        
# Convert the target to 0s and 1s
lb = LabelBinarizer()
y = lb.fit_transform(df['Churn'])

In [8]:
train.dtypes

gender                int32
SeniorCitizen         int32
Partner               int32
Dependents            int32
tenure                int64
PhoneService          int32
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling      int32
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
dtype: object

In [9]:
# Now we deal with the blank spaces in TotalCharges

# convert spaces to np.Nan so we can cast the column to float
train['TotalCharges'] = train['TotalCharges'].replace(" ", np.nan)

train['TotalCharges'] = train['TotalCharges'].astype(float)

train['TotalCharges'].replace(np.nan, train['TotalCharges'].median(), inplace=True)
train['TotalCharges'].value_counts()

1397.475    11
20.200      11
19.750       9
19.650       8
20.050       8
            ..
607.300      1
1534.050     1
1066.150     1
249.950      1
1024.000     1
Name: TotalCharges, Length: 6531, dtype: int64

In [10]:
train.dtypes

gender                int32
SeniorCitizen         int32
Partner               int32
Dependents            int32
tenure                int64
PhoneService          int32
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling      int32
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
dtype: object

In [11]:
train.shape

(7043, 19)

In [12]:
# sort the dataframe columns by data type

col_types = train.columns.to_series().groupby(train.dtypes).groups
col_types

{dtype('int32'): Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
        'PaperlessBilling'],
       dtype='object'),
 dtype('int64'): Index(['tenure'], dtype='object'),
 dtype('float64'): Index(['MonthlyCharges', 'TotalCharges'], dtype='object'),
 dtype('O'): Index(['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
        'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
        'Contract', 'PaymentMethod'],
       dtype='object')}

In [13]:
# change the dictionary keys from dtype objects to strings

col_types = {k.name: v for k, v in col_types.items()}
col_types

{'int32': Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
        'PaperlessBilling'],
       dtype='object'),
 'int64': Index(['tenure'], dtype='object'),
 'float64': Index(['MonthlyCharges', 'TotalCharges'], dtype='object'),
 'object': Index(['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
        'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
        'Contract', 'PaymentMethod'],
       dtype='object')}

In [14]:
col_types['object']

Index(['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaymentMethod'],
      dtype='object')

In [15]:
#import category_encoders as ce  <- don't have this package installed :/

from sklearn.preprocessing import OneHotEncoder

onehot= OneHotEncoder()

onehot_array = onehot.fit_transform(train[col_types['object']]).toarray()

In [16]:
# We had 9 features with 3 cols and 1 feature with 4. 
# We should get 31 columns

onehot_array.shape

(7043, 31)

In [17]:
onehot_array

array([[0., 1., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [18]:
# Now we convert the rest of the data to an array so we can combine with the encoded array
array_data = train.drop(columns = col_types['object'], axis = 1)
array_data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges
0,0,0,1,0,1,0,1,29.85,29.85
1,1,0,0,0,34,1,0,56.95,1889.50
2,1,0,0,0,2,1,1,53.85,108.15
3,1,0,0,0,45,0,0,42.30,1840.75
4,0,0,0,0,2,1,1,70.70,151.65


In [19]:
array_data = array_data.to_numpy()
array_data

array([[0.0000e+00, 0.0000e+00, 1.0000e+00, ..., 1.0000e+00, 2.9850e+01,
        2.9850e+01],
       [1.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 5.6950e+01,
        1.8895e+03],
       [1.0000e+00, 0.0000e+00, 0.0000e+00, ..., 1.0000e+00, 5.3850e+01,
        1.0815e+02],
       ...,
       [0.0000e+00, 0.0000e+00, 1.0000e+00, ..., 1.0000e+00, 2.9600e+01,
        3.4645e+02],
       [1.0000e+00, 1.0000e+00, 1.0000e+00, ..., 1.0000e+00, 7.4400e+01,
        3.0660e+02],
       [1.0000e+00, 0.0000e+00, 0.0000e+00, ..., 1.0000e+00, 1.0565e+02,
        6.8445e+03]])

In [20]:
numeric = array_data[:, [4,7,8]]

numeric = numeric.astype(str)


float_array = []

for row in numeric:

    n = np.array([v.replace(',', '') for v in row], dtype=np.float32)
    float_array.append(n)

float_array = np.array(float_array)
float_array

array([[1.0000e+00, 2.9850e+01, 2.9850e+01],
       [3.4000e+01, 5.6950e+01, 1.8895e+03],
       [2.0000e+00, 5.3850e+01, 1.0815e+02],
       ...,
       [1.1000e+01, 2.9600e+01, 3.4645e+02],
       [4.0000e+00, 7.4400e+01, 3.0660e+02],
       [6.6000e+01, 1.0565e+02, 6.8445e+03]], dtype=float32)

In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_numerics = scaler.fit_transform(float_array)
scaled_numerics

array([[-1.2774446 , -1.1603229 , -0.99424195],
       [ 0.06632742, -0.25962892, -0.17324413],
       [-1.2367243 , -0.36266038, -0.95967406],
       ...,
       [-0.8702409 , -1.1686319 , -0.8544695 ],
       [-1.1552835 ,  0.32033828, -0.8720624 ],
       [ 1.369379  ,  1.3589613 ,  2.014288  ]], dtype=float32)

In [22]:
# After dropping the ID and target features, we had 19 columns
# 10 were one-hot encoded. 
# So number of columns in final array should be 19 - 10 + 31 = 40

cleaned_data = np.concatenate((array_data[:, [0,1,2,3,5,6]], scaled_numerics, onehot_array), axis=1)

cleaned_data.shape

(7043, 40)

## Time to make the model

In [ ]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam

# fix random seed for reproducibility
seed = 42
numpy.random.seed(seed)

# Optimization function
lr = 0.001
Opt_function = Adam(learning_rate = lr)

# Function to create model, required for KerasClassifier
def create_model(lr = 0.001):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=40, activation='selu', kernel_initializer='lecun_normal'))
    model.add(Dense(12, activation='selu', kernel_initializer='lecun_normal'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=Opt_function, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [32, 64, 128],
              'lr':[0.001],
              'epochs': [200]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(cleaned_data, y)

# Report Results
print(f"\nBest: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

In [31]:
lr= ""

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?